# Streamlit NLP Suite — Sentiment, NER, Summarization, Keywords
Run classic NLP tasks via Transformers pipelines or LLM providers (OpenAI, Gemini, Anthropic, Groq, HF Inference) from a simple Streamlit UI.


# Installation (commented)

In [ ]:
# !pip install streamlit transformers sentence-transformers spacy openai anthropic google-generativeai groq huggingface_hub
# !python -m spacy download en_core_web_sm  # optional for local NER


# Imports

In [ ]:
import os
import streamlit as st


# Helper: provider calls (LLM-based)

In [ ]:
def llm_complete(task_prompt: str, provider: str, model: str, temperature: float, max_tokens: int) -> str:
    if provider == 'OpenAI':
        from openai import OpenAI
        if (k:=st.secrets.get('OPENAI_API_KEY', None) if hasattr(st,'secrets') else None) or os.environ.get('OPENAI_API_KEY'):
            os.environ['OPENAI_API_KEY'] = k or os.environ.get('OPENAI_API_KEY','')
        client = OpenAI()
        out = client.chat.completions.create(model=model, temperature=temperature, max_tokens=max_tokens,
                                             messages=[{"role":"system","content":"You are an NLP assistant."}, {"role":"user","content":task_prompt}])
        return out.choices[0].message.content
    if provider == 'Gemini':
        import google.generativeai as genai
        genai.configure(api_key=st.secrets.get('GEMINI_API_KEY', os.environ.get('GEMINI_API_KEY')))
        g = genai.GenerativeModel(model)
        res = g.generate_content(task_prompt)
        return getattr(res,'text',str(res))
    if provider == 'Anthropic':
        import anthropic
        a = anthropic.Anthropic()
        res = a.messages.create(model=model, max_tokens=max_tokens, temperature=temperature, messages=[{"role":"user","content":task_prompt}])
        return res.content[0].text
    if provider == 'Groq':
        from groq import Groq
        gq = Groq(api_key=st.secrets.get('GROQ_API_KEY', os.environ.get('GROQ_API_KEY')))
        res = gq.chat.completions.create(model=model, temperature=temperature, max_tokens=max_tokens,
                                         messages=[{"role":"system","content":"You are an NLP assistant."}, {"role":"user","content":task_prompt}])
        return res.choices[0].message.content
    if provider == 'HuggingFace':
        from huggingface_hub import InferenceClient
        hf = InferenceClient(token=st.secrets.get('HUGGINGFACEHUB_API_TOKEN', os.environ.get('HUGGINGFACEHUB_API_TOKEN')))
        try:
            res = hf.chat_completion(model=model, messages=[{"role":"user","content":task_prompt}], max_tokens=max_tokens)
            return res.choices[0].message['content'] if hasattr(res.choices[0],'message') else res.choices[0]['message']['content']
        except Exception:
            return hf.text_generation(model=model, inputs=task_prompt, max_new_tokens=max_tokens, temperature=temperature)
    raise ValueError('Unsupported provider')


# Helper: transformers pipelines (locally)

In [ ]:
def get_pipeline(task: str, model: str | None = None):
    from transformers import pipeline
    return pipeline(task) if not model else pipeline(task, model=model)


# UI

In [ ]:
st.set_page_config(page_title="NLP Suite", page_icon="🧠")
st.title("🧠 NLP Suite — Transformers & LLMs")

with st.sidebar:
    backend = st.selectbox('Backend', ['Transformers (local)', 'LLM Provider'], index=0)
    if backend == 'LLM Provider':
        provider = st.selectbox('Provider', ['OpenAI','Gemini','Anthropic','Groq','HuggingFace'], index=0)
        if provider=='OpenAI':
            model = st.text_input('Model', 'gpt-4o-mini')
        elif provider=='Gemini':
            model = st.text_input('Model', 'gemini-1.5-flash')
        elif provider=='Anthropic':
            model = st.text_input('Model', 'claude-3-5-sonnet-20241022')
        elif provider=='Groq':
            model = st.text_input('Model', 'llama-3.1-8b-instant')
        else:
            model = st.text_input('Model', 'Qwen/Qwen2.5-1.5B-Instruct')
        temperature = st.slider('temperature', 0.0, 1.5, 0.2, 0.1)
        max_tokens = st.slider('max tokens', 64, 2048, 256, 32)
    else:
        provider = model = None
        temperature = 0.0
        max_tokens = 256

tab1, tab2, tab3, tab4 = st.tabs(["Sentiment", "NER", "Summarization", "Keywords"])

with tab1:
    st.subheader('Sentiment Analysis')
    text = st.text_area('Input text', height=120)
    if st.button('Analyze sentiment', key='sentiment_btn') and text.strip():
        if backend.startswith('Transformers'):
            clf = get_pipeline('sentiment-analysis')
            res = clf(text)
            st.json(res)
        else:
            prompt = f"Classify sentiment as POSITIVE or NEGATIVE with confidence.\nText: {text}"
            out = llm_complete(prompt, provider, model, temperature, max_tokens)
            st.write(out)

with tab2:
    st.subheader('Named Entity Recognition')
    text2 = st.text_area('Input text for NER', height=120)
    if st.button('Extract entities', key='ner_btn') and text2.strip():
        if backend.startswith('Transformers'):
            ner = get_pipeline('ner')
            res = ner(text2)
            st.json(res)
        else:
            prompt = """Extract named entities from the text and return JSON with keys: person, org, location, date, other.\nText: """ + text2
            out = llm_complete(prompt, provider, model, temperature, max_tokens)
            st.write(out)

with tab3:
    st.subheader('Summarization')
    text3 = st.text_area('Input text to summarize', height=160)
    if st.button('Summarize', key='sum_btn') and text3.strip():
        if backend.startswith('Transformers'):
            summ = get_pipeline('summarization')
            res = summ(text3)
            st.json(res)
        else:
            prompt = f"Summarize the following text in 3-5 bullet points.\n\n{text3}"
            out = llm_complete(prompt, provider, model, temperature, max_tokens)
            st.write(out)

with tab4:
    st.subheader('Keyword Extraction')
    text4 = st.text_area('Input text for keywords', height=140)
    if st.button('Extract keywords', key='kw_btn') and text4.strip():
        if backend.startswith('Transformers'):
            # Simple local heuristic via TextRank or KeyBERT could be added; here, use a small prompt to a local model if available.
            try:
                from transformers import pipeline
                gen = pipeline('text-generation', model='gpt2')
                res = gen(f"Extract 5-10 keywords: {text4}", max_new_tokens=30)
                st.json(res)
            except Exception as e:
                st.warning(f"Local keyword extraction not configured: {e}")
        else:
            prompt = f"Extract 5-10 keywords (comma-separated) capturing main topics from the text.\nText: {text4}"
            out = llm_complete(prompt, provider, model, temperature, max_tokens)
            st.write(out)


# Notes
# - Transformers pipelines may download models on first run.
# - For production, prefer deterministic prompts and JSON outputs with schema validation.